## **Firmas Digitales**

Tres casos:
1) Documento modificado.
2) Firma no corresponde
3) Deje todo correcto: doc OK y firma OK


**Código Base (Común para todos los casos)**

In [ ]:
import os
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet
import base64
import json
from datetime import datetime

# Función para generar claves asimétricas
def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )

# Función para crear documento
def crear_documento(autor, titulo, contenido, nombre_archivo="documento.json"):
    """
    Crea un documento en formato JSON para ser firmado.
    """
    documento = {
        "autor": autor,
        "titulo": titulo,
        "contenido": contenido,
        "fecha_creacion": datetime.now().isoformat()
    }

    with open(nombre_archivo, "w") as f:
        json.dump(documento, f, indent=4)

    print(f"Documento creado y guardado en: {nombre_archivo}")
    return documento

# Función para firmar documento
def firmar_documento(ruta_documento, ruta_firma="firma_digital.bin"):
    """
    Firma digitalmente un documento utilizando una clave privada RSA.
    """
    # Cargamos la clave privada
    private_key = cargar_clave_privada()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()

    # Firmamos el mensaje
    firma = private_key.sign(
        mensaje,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    # Guardamos la firma en un archivo
    with open(ruta_firma, "wb") as f:
        f.write(firma)

    print(f"Documento firmado. Firma guardada en: {ruta_firma}")
    return firma

# Función para verificar firma
def verificar_firma(ruta_documento, ruta_firma):
    """
    Verifica la firma digital de un documento utilizando una clave pública RSA.
    """
    # Cargamos la clave pública
    public_key = cargar_clave_publica()

    # Cargamos la firma
    with open(ruta_firma, "rb") as f:
        firma = f.read()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()

    try:
        # Verificamos la firma
        public_key.verify(
            firma,
            mensaje,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("¡La firma es válida! El documento es auténtico y no ha sido modificado.")
        return True
    except Exception as e:
        print("¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.")
        print(f"Error: {e}")
        return False

# **Caso 1**

In [ ]:
# Generamos las claves
generar_claves_asimetricas()

# Creamos y firmamos el documento original
print("\n--- Creando documento original ---")
documento_original = crear_documento(
    "Juan Pérez",
    "Declaración de Seguridad",
    "Este documento contiene información confidencial sobre vulnerabilidades de seguridad.",
    "documento_original.json"
)

# Firmamos el documento original
print("\n--- Firmando documento original ---")
firmar_documento("documento_original.json", "firma_original.bin")

# Modificamos el documento (simulando un cambio)
print("\n--- Modificando el documento ---")
with open("documento_original.json", "r") as f:
    doc = json.load(f)
doc["contenido"] = "Este documento HA SIDO MODIFICADO y ahora contiene información falsa."
with open("documento_modificado.json", "w") as f:
    json.dump(doc, f, indent=4)

# Intentamos verificar la firma original con el documento modificado
print("\n--- Verificando firma con documento modificado ---")
verificacion = verificar_firma("documento_modificado.json", "firma_original.bin")

print("\nResultado de la verificación:", "VÁLIDA" if verificacion else "INVÁLIDA (como se esperaba)")

Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.

--- Creando documento original ---
Documento creado y guardado en: documento_original.json

--- Firmando documento original ---
Documento firmado. Firma guardada en: firma_original.bin

--- Modificando el documento ---

--- Verificando firma con documento modificado ---
¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.
Error: 

Resultado de la verificación: INVÁLIDA (como se esperaba)


# **Caso 2**

In [ ]:
# Generamos las claves
generar_claves_asimetricas()

# Creamos dos documentos diferentes
print("\n--- Creando documentos ---")
documento1 = crear_documento(
    "Ana García",
    "Informe Técnico 1",
    "Contenido del primer informe técnico.",
    "documento1.json"
)

documento2 = crear_documento(
    "Carlos López",
    "Informe Técnico 2",
    "Contenido del segundo informe técnico.",
    "documento2.json"
)

# Firmamos solo el primer documento
print("\n--- Firmando solo el primer documento ---")
firma1 = firmar_documento("documento1.json", "firma1.bin")

# Intentamos verificar el segundo documento con la firma del primero
print("\n--- Intentando verificar documento2 con firma1 ---")
verificacion = verificar_firma("documento2.json", "firma1.bin")

print("\nResultado de la verificación:", "VÁLIDA" if verificacion else "INVÁLIDA (como se esperaba)")

Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.

--- Creando documentos ---
Documento creado y guardado en: documento1.json
Documento creado y guardado en: documento2.json

--- Firmando solo el primer documento ---
Documento firmado. Firma guardada en: firma1.bin

--- Intentando verificar documento2 con firma1 ---
¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.
Error: 

Resultado de la verificación: INVÁLIDA (como se esperaba)


# **Caso 3**

In [ ]:
# Generamos las claves
generar_claves_asimetricas()

# Creamos un documento
print("\n--- Creando documento ---")
documento = crear_documento(
    "María Rodríguez",
    "Reporte de Auditoría",
    "Resultados de la auditoría de seguridad realizada el mes pasado.",
    "documento_correcto.json"
)

# Firmamos el documento
print("\n--- Firmando documento ---")
firma = firmar_documento("documento_correcto.json", "firma_correcta.bin")

# Verificamos la firma con el documento original
print("\n--- Verificando firma ---")
verificacion = verificar_firma("documento_correcto.json", "firma_correcta.bin")

print("\nResultado de la verificación:", "VÁLIDA" if verificacion else "INVÁLIDA")

Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.

--- Creando documento ---
Documento creado y guardado en: documento_correcto.json

--- Firmando documento ---
Documento firmado. Firma guardada en: firma_correcta.bin

--- Verificando firma ---
¡La firma es válida! El documento es auténtico y no ha sido modificado.

Resultado de la verificación: VÁLIDA


En el error hay que evaluar esas 3 opciones porque la otra es cuando todo funciona, para la otra clase.

y hacer los 5 ejercicios individualmente haciendo la simulacines como si tú tuvieras varios roles.

Programa de **FIRMAS DIGITALES** todo en uno.

In [2]:
import os
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
import json
from datetime import datetime
from IPython.display import display, Markdown

# --- Configuración inicial para Google Colab ---
# Crear archivos de ejemplo automáticamente si no existen
def setup_colab_environment():
    """Configura el entorno con documentos de ejemplo para Google Colab"""
    if not os.path.exists("clave_privada.pem"):
        generar_claves_asimetricas()

    # Documento original
    if not os.path.exists("documento_original.json"):
        crear_documento("Juan Pérez", "Contrato importante",
                       "Este es el contenido original del contrato firmado.",
                       "documento_original.json")

    # Documento modificado
    if not os.path.exists("documento_modificado.json"):
        crear_documento("Juan Pérez", "Contrato importante",
                       "Este es el contenido MODIFICADO del contrato.",
                       "documento_modificado.json")

    # Firmar el documento original si no existe firma
    if not os.path.exists("firma_valida.bin"):
        firmar_documento("documento_original.json", "firma_valida.bin")

    # Crear firma inválida (firmando el documento modificado)
    if not os.path.exists("firma_invalida.bin"):
        firmar_documento("documento_modificado.json", "firma_invalida.bin")

# --- Funciones principales (las que ya tenías, mejor comentadas) ---

def generar_claves_asimetricas():
    """Genera un par de claves RSA (privada y pública) y las guarda en archivos PEM"""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as f:
        f.write(clave_privada.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as f:
        f.write(clave_publica.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

    display(Markdown("**Claves generadas:** Se crearon 'clave_privada.pem' y 'clave_publica.pem'"))

def cargar_clave_privada():
    """Carga la clave privada desde archivo PEM"""
    with open("clave_privada.pem", "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None)

def cargar_clave_publica():
    """Carga la clave pública desde archivo PEM"""
    with open("clave_publica.pem", "rb") as f:
        return serialization.load_pem_public_key(f.read())

def crear_documento(autor, titulo, contenido, nombre_archivo):
    """Crea un documento JSON con metadatos y contenido"""
    documento = {
        "autor": autor,
        "titulo": titulo,
        "contenido": contenido,
        "fecha_creacion": datetime.now().isoformat()
    }
    with open(nombre_archivo, "w") as f:
        json.dump(documento, f, indent=4)
    display(Markdown(f"**Documento creado:** '{nombre_archivo}'"))
    return documento

def firmar_documento(ruta_documento, ruta_firma):
    """Firma digitalmente un documento usando la clave privada"""
    private_key = cargar_clave_privada()

    with open(ruta_documento) as f:
        mensaje = json.dumps(json.load(f)).encode()

    firma = private_key.sign(
        mensaje,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    with open(ruta_firma, "wb") as f:
        f.write(firma)

    display(Markdown(f"**Firma creada:** '{ruta_firma}' para documento '{ruta_documento}'"))

# --- Nuevas funciones para verificación ---

def verificar_integridad(documento1, documento2):
    """Compara dos documentos usando hash SHA-256 para detectar modificaciones"""
    with open(documento1, "rb") as f1, open(documento2, "rb") as f2:
        hash1 = hashlib.sha256(f1.read()).hexdigest()
        hash2 = hashlib.sha256(f2.read()).hexdigest()

    if hash1 == hash2:
        display(Markdown("**✅ Integridad:** Los documentos son idénticos"))
        return True
    else:
        display(Markdown("**❌ Integridad:** Los documentos son diferentes"))
        return False

def verificar_firma(ruta_documento, ruta_firma):
    """Verifica que la firma corresponda al documento usando la clave pública"""
    public_key = cargar_clave_publica()

    with open(ruta_firma, "rb") as f:
        firma = f.read()

    with open(ruta_documento) as f:
        mensaje = json.dumps(json.load(f)).encode()

    try:
        public_key.verify(
            firma,
            mensaje,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        display(Markdown("**✅ Firma válida:** Corresponde al documento"))
        return True
    except Exception as e:
        display(Markdown(f"**❌ Firma inválida:** No corresponde al documento\nError: `{str(e)}`"))
        return False

# --- Menú interactivo ---

def mostrar_menu():
    """Muestra un menú interactivo con las opciones de verificación"""
    setup_colab_environment()  # Configura el entorno con archivos de ejemplo

    while True:
        display(Markdown("""
## 🛡️ Validador de Firmas Digitales

**Opciones:**
1. Verificar si un documento fue modificado
2. Verificar si una firma es válida
3. Verificar documento y firma (completo)
4. Mostrar documentos de ejemplo
5. Salir
"""))

        opcion = input("Seleccione una opción (1-5): ")

        if opcion == "1":
            display(Markdown("### 🔍 Verificar integridad del documento"))
            doc1 = input("Ruta del documento ORIGINAL (ej. documento_original.json): ") or "documento_original.json"
            doc2 = input("Ruta del documento a COMPARAR (ej. documento_modificado.json): ") or "documento_modificado.json"
            verificar_integridad(doc1, doc2)

        elif opcion == "2":
            display(Markdown("### 🔏 Verificar validez de firma"))
            doc = input("Ruta del documento (ej. documento_original.json): ") or "documento_original.json"
            firma = input("Ruta de la firma (ej. firma_valida.bin): ") or "firma_valida.bin"
            verificar_firma(doc, firma)

        elif opcion == "3":
            display(Markdown("### 🔐 Verificación completa"))
            doc_orig = input("Ruta del documento ORIGINAL (ej. documento_original.json): ") or "documento_original.json"
            doc_verif = input("Ruta del documento a VERIFICAR (ej. documento_modificado.json): ") or "documento_modificado.json"
            firma = input("Ruta de la firma (ej. firma_valida.bin): ") or "firma_valida.bin"

            display(Markdown("**1. Verificando integridad...**"))
            integridad_ok = verificar_integridad(doc_orig, doc_verif)

            display(Markdown("**2. Verificando firma...**"))
            firma_ok = verificar_firma(doc_verif, firma)

            display(Markdown("### 📝 Resultado final"))
            if integridad_ok and firma_ok:
                display(Markdown("**✅ Todo correcto:** Documento íntegro y firma válida"))
            elif not integridad_ok and firma_ok:
                display(Markdown("**⚠️ Caso raro:** Documento modificado pero firma válida (¿firmó el documento modificado?)"))
            elif integridad_ok and not firma_ok:
                display(Markdown("**❌ Firma inválida:** Documento no modificado pero firma no corresponde"))
            else:
                display(Markdown("**❌❌ Problemas graves:** Documento modificado y firma inválida"))

        elif opcion == "4":
            display(Markdown("### 📄 Documentos de ejemplo"))
            display(Markdown("**Documento original (válido):** `documento_original.json`"))
            display(Markdown("**Documento modificado:** `documento_modificado.json`"))
            display(Markdown("**Firma válida:** `firma_valida.bin` (para documento original)"))
            display(Markdown("**Firma inválida:** `firma_invalida.bin` (para documento modificado)"))

            # Mostrar contenido de los documentos
            with open("documento_original.json") as f:
                display(Markdown(f"```json\n{f.read()}\n```"))

        elif opcion == "5":
            display(Markdown("**👋 Saliendo del validador...**"))
            break

        else:
            display(Markdown("**⚠️ Opción no válida. Por favor elija 1-5**"))

        input("\nPresione Enter para continuar...")
        display(Markdown("---"))

# --- Ejecución en Colab ---
display(Markdown("# 🔐 Validador de Firmas Digitales en Google Colab"))
mostrar_menu()

# 🔐 Validador de Firmas Digitales en Google Colab

**Documento creado:** 'documento_original.json'

**Documento creado:** 'documento_modificado.json'

**Firma creada:** 'firma_valida.bin' para documento 'documento_original.json'

**Firma creada:** 'firma_invalida.bin' para documento 'documento_modificado.json'

...


Presione Enter para continuar...


---


## 🛡️ Validador de Firmas Digitales

**Opciones:**
1. Verificar si un documento fue modificado
2. Verificar si una firma es válida
3. Verificar documento y firma (completo)
4. Mostrar documentos de ejemplo
5. Salir


Seleccione una opción (1-5): 5


**👋 Saliendo del validador...**

EJEMPLO DE CODIGO REAL PARA  FIRMAS DIGITALES dDE DOCUMENTOS EN TU EQUIPO.

In [1]:
import os
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet
import base64
import json
from datetime import datetime

# Función para generar claves asimétricas
def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )

# Función para crear documento
def crear_documento(autor, titulo, contenido, nombre_archivo="documento.json"):
    """
    Crea un documento en formato JSON para ser firmado.
    """
    documento = {
        "autor": autor,
        "titulo": titulo,
        "contenido": contenido,
        "fecha_creacion": datetime.now().isoformat()
    }

    with open(nombre_archivo, "w") as f:
        json.dump(documento, f, indent=4)

    print(f"Documento creado y guardado en: {nombre_archivo}")
    return documento

# Función para firmar documento
def firmar_documento(ruta_documento, ruta_firma="firma_digital.bin"):
    """
    Firma digitalmente un documento utilizando una clave privada RSA.
    """
    # Cargamos la clave privada
    private_key = cargar_clave_privada()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()

    # Firmamos el mensaje
    firma = private_key.sign(
        mensaje,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    # Guardamos la firma en un archivo
    with open(ruta_firma, "wb") as f:
        f.write(firma)

    print(f"Documento firmado. Firma guardada en: {ruta_firma}")
    return firma

# Función para verificar firma
def verificar_firma(ruta_documento, ruta_firma):
    """
    Verifica la firma digital de un documento utilizando una clave pública RSA.
    """
    # Cargamos la clave pública
    public_key = cargar_clave_publica()

    # Cargamos la firma
    with open(ruta_firma, "rb") as f:
        firma = f.read()

    # Convertimos el documento a una cadena JSON y la codificamos en bytes
    documento = open(ruta_documento)
    mensaje = json.load(documento)
    mensaje = json.dumps(mensaje).encode()
    documento.close()

    try:
        # Verificamos la firma
        public_key.verify(
            firma,
            mensaje,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("¡La firma es válida! El documento es auténtico y no ha sido modificado.")
        return True
    except Exception as e:
        print("¡La firma NO es válida! El documento puede haber sido alterado o la firma no corresponde.")
        print(f"Error: {e}")
        return False

# Función para verificar integridad del documento
def verificar_integridad_documento(ruta_documento_original, ruta_documento_verificar):
    """
    Verifica si el contenido de dos documentos es idéntico comparando sus hashes.
    """
    try:
        with open(ruta_documento_original, "rb") as f:
            hash_original = hashlib.sha256(f.read()).hexdigest()

        with open(ruta_documento_verificar, "rb") as f:
            hash_verificar = hashlib.sha256(f.read()).hexdigest()

        if hash_original == hash_verificar:
            print("Documento OK: El documento no ha sido modificado.")
            return True
        else:
            print("Documento MODIFICADO: El contenido ha cambiado.")
            return False
    except Exception as e:
        print(f"Error al verificar documentos: {e}")
        return False

# Función para mostrar el menú principal
def mostrar_menu():
    """
    Muestra el menú de opciones y maneja la interacción con el usuario.
    """
    while True:
        print("\n--- Menú de Validación de Firmas Digitales ---")
        print("1. Verificar integridad del documento")
        print("2. Verificar firma digital")
        print("3. Verificar documento y firma")
        print("4. Salir")

        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            # Verificar solo integridad del documento
            doc_original = input("Ruta del documento original: ")
            doc_verificar = input("Ruta del documento a verificar: ")
            verificar_integridad_documento(doc_original, doc_verificar)

        elif opcion == "2":
            # Verificar solo la firma
            doc = input("Ruta del documento: ")
            firma = input("Ruta de la firma: ")
            verificar_firma(doc, firma)

        elif opcion == "3":
            # Verificar ambos: documento y firma
            doc_original = input("Ruta del documento original: ")
            doc_verificar = input("Ruta del documento a verificar: ")
            firma = input("Ruta de la firma: ")

            # Primero verificamos el documento
            doc_ok = verificar_integridad_documento(doc_original, doc_verificar)

            # Luego verificamos la firma
            firma_ok = verificar_firma(doc_verificar, firma)

            # Mostramos resumen final
            print("\n--- Resumen de Verificación ---")
            if doc_ok and firma_ok:
                print("ESTADO: Todo correcto - Documento OK y Firma OK")
            elif not doc_ok and firma_ok:
                print("ESTADO: Documento modificado pero firma válida (raro caso)")
            elif doc_ok and not firma_ok:
                print("ESTADO: Documento no modificado pero firma no válida")
            else:
                print("ESTADO: Documento modificado y firma no válida")

        elif opcion == "4":
            print("Saliendo del programa...")
            break

        else:
            print("Opción no válida. Por favor seleccione 1-4.")

# Función principal
def main():
    """
    Función principal que inicia la aplicación.
    """
    # Verificamos si existen las claves, si no, las generamos
    if not os.path.exists("clave_privada.pem") or not os.path.exists("clave_publica.pem"):
        print("Generando claves asimétricas...")
        generar_claves_asimetricas()

    # Mostramos el menú
    mostrar_menu()

if __name__ == "__main__":
    main()

Generando claves asimétricas...
Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.

--- Menú de Validación de Firmas Digitales ---
1. Verificar integridad del documento
2. Verificar firma digital
3. Verificar documento y firma
4. Salir
Seleccione una opción: 1
Ruta del documento original: doc_original
Ruta del documento a verificar: doc_verificar
Error al verificar documentos: [Errno 2] No such file or directory: 'doc_original'

--- Menú de Validación de Firmas Digitales ---
1. Verificar integridad del documento
2. Verificar firma digital
3. Verificar documento y firma
4. Salir
Seleccione una opción: 4
Saliendo del programa...
